# List of publications

In [4]:
import re
import requests
from IPython.display import HTML

# === CONFIG ===
ORCID_ID = "0000-0002-0705-7662"   # your ORCID iD
API_BASE = "https://pub.orcid.org/v3.0"
# ==============

session = requests.Session()
session.headers.update({"Accept": "application/json"})


def get_all_put_codes(orcid: str):
    url = f"{API_BASE}/{orcid}/works"
    r = session.get(url, timeout=30)
    r.raise_for_status()
    works = r.json()

    put_codes = []
    for group in works.get("group", []):
        for ws in group.get("work-summary", []):
            put_codes.append(ws["put-code"])
    return put_codes


def get_work_detail(orcid: str, put_code: int):
    url = f"{API_BASE}/{orcid}/work/{put_code}"
    r = session.get(url, timeout=30)
    r.raise_for_status()
    return r.json()


def _norm(s):
    return re.sub(r"\s+", " ", (s or "").strip())


def extract_external_ids(work):
    ext = (work.get("external-ids") or {}).get("external-id") or []
    out = {}
    for e in ext:
        t = (e.get("external-id-type") or "").lower()
        v = e.get("external-id-value") or ""
        u = ((e.get("external-id-url") or {}).get("value")) or ""
        if t and v:
            out[t] = u or v
    return out


def extract_contributors(work):
    contribs = (work.get("contributors") or {}).get("contributor") or []
    people = []
    for c in contribs:
        name = ((c.get("credit-name") or {}).get("value")) or ""
        attrs = c.get("contributor-attributes") or {}
        seq = attrs.get("contributor-sequence") or ""
        people.append({"name": _norm(name), "seq": seq})

    first = [p for p in people if p["seq"] == "FIRST"]
    additional = [p for p in people if p["seq"] != "FIRST"]
    ordered = first + additional

    return [p["name"] for p in ordered if p["name"]]


def name_to_apa(full_name: str) -> str:
    parts = [p for p in (full_name or "").split() if p]
    if len(parts) == 0:
        return ""
    if len(parts) == 1:
        return parts[0]

    last = parts[-1]
    initials = []
    for p in parts[:-1]:
        ch = re.sub(r"[^A-Za-zÀ-ÖØ-öø-ÿ]", "", p)[:1]
        if ch:
            initials.append(ch.upper() + ".")
    return f"{last}, {' '.join(initials)}".strip()


def format_authors_apa(authors):
    if not authors:
        return ""

    apa = [name_to_apa(a) for a in authors if a]
    apa = [a for a in apa if a]

    if len(apa) == 0:
        return ""
    if len(apa) <= 2:
        return " & ".join(apa)
    if len(apa) <= 20:
        return ", ".join(apa[:-1]) + ", & " + apa[-1]

    first_19 = apa[:19]
    last = apa[-1]
    return ", ".join(first_19) + ", …, " + last


def extract_info(work):
    title_obj = work.get("title") or {}
    title = _norm((title_obj.get("title") or {}).get("value"))
    subtitle = _norm((title_obj.get("subtitle") or {}).get("value"))
    if subtitle:
        title = f"{title}: {subtitle}" if title else subtitle

    journal = _norm((work.get("journal-title") or {}).get("value"))

    pub_date = work.get("publication-date") or {}
    year = ((pub_date.get("year") or {}).get("value")) or None

    ext = extract_external_ids(work)
    doi = ext.get("doi")
    doi_url = None
    if doi:
        doi_url = doi if str(doi).startswith("http") else f"https://doi.org/{doi}"

    fallback_url = ext.get("url") or ext.get("uri")
    if fallback_url and not str(fallback_url).startswith("http"):
        fallback_url = None

    authors = extract_contributors(work)

    return {
        "authors": authors,
        "year": year,
        "title": title,
        "journal": journal,
        "doi_url": doi_url,
        "fallback_url": fallback_url,
    }


def apa_reference(rec):
    authors_str = format_authors_apa(rec["authors"])
    year = rec["year"] or "n.d."
    title = rec["title"] or "Untitled work"
    journal = rec["journal"]

    title_part = f"{title}."
    journal_part = f"{journal}. " if journal else ""
    link = rec["doi_url"] or rec["fallback_url"] or ""

    if authors_str:
        ref = f"{authors_str} ({year}). {title_part} {journal_part}{link}".strip()
    else:
        ref = f"{title_part} ({year}). {journal_part}{link}".strip()

    return re.sub(r"\s{2,}", " ", ref)


def render_apa_list(records):
    items = []
    for rec in records:
        ref = apa_reference(rec)

        link = rec["doi_url"] or rec["fallback_url"]
        if link:
            ref = ref.replace(link, f'<a href="{link}" target="_blank">{link}</a>')

        items.append(f"<li style='margin: 0.4em 0;'>{ref}</li>")

    html = f"""
    <div style="line-height:1.55;">
      <ol style="padding-left: 1.3em; margin: 0.2em 0;">
        {''.join(items)}
      </ol>
    </div>
    """
    return HTML(html)


# === RUN (NO display() here; we store the result for Cell 2) ===
put_codes = get_all_put_codes(ORCID_ID)
records = [extract_info(get_work_detail(ORCID_ID, pc)) for pc in put_codes]

def sort_key(r):
    y = r["year"]
    return int(y) if (y and str(y).isdigit()) else -1

records = sorted(records, key=sort_key, reverse=True)

pubs_html = render_apa_list(records)  # Cell 2 should: display(pubs_html)


In [6]:
from IPython.display import display
display(pubs_html)